# CAPSTONE:

# SpaceX Falcon 9 First Stage Landing Prediction

## Lab 5:

## Launch Sites Locations Analysis with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories.

Finding an optimal location for building a launch site certainly involves many factors and hopefully we will discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn`, and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

## Tasks and Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map;
*   **TASK 2:** Mark the success/failed launches for each site on the map;
*   **TASK 3:** Calculate the distances between a launch site to its proximities.

After completing the tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
# Import required libraries:

import folium
import pandas as pd

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

### Task 1: Mark all launch sites on a map

First, let's add each site's location on a map using site's latitude and longitude coordinates

The file `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [2]:
# Download and read the `spacex_launch_geo.csv`

!curl -o spacex_csv_file https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv

spacex_df=pd.read_csv('spacex_csv_file')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7710  100  7710    0     0  14452      0 --:--:-- --:--:-- --:--:-- 14465


Now, you can take a look at the coordinates for each site.

In [3]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that cannot give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

site_map

We can use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

site_map

You should find a small yellow circle near the city of Houston, and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in dataframe `launch_sites`

**TO DO:** Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map.

*An example of folium.Circle*:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

*An example of folium.Marker*:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [7]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. 
# Add Launch Site name as a popup label

The generated map with marked launch sites should look similar to the following:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>

In [8]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Instantiate a launch sites dictionary
launch_sites = [
    {'name': 'CCAFS LC', 'coordinates': [28.562302, -80.577356], 'popup': 'Cape Canaveral LC'},
    {'name': 'CCAFS SLC', 'coordinates': [28.563197, -80.576820], 'popup': 'Cape Canaveral SLC'},
    {'name': 'KSC LC', 'coordinates': [28.573255, -80.646895], 'popup': 'Kennedy Space Center LC'},
    {'name': 'VAFB SLC', 'coordinates': [34.632834, -120.610745], 'popup': 'Vandenberg Space Launch Complex'}
]

# Add circles and markers for all launch sites
for site in launch_sites:
    name = site['name']
    coordinates = site['coordinates']
    popup = site['popup']
    
    # Add circle
    folium.Circle(
        location=coordinates,
        radius=1000,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=popup
    ).add_to(site_map)
    
    # Add marker
    folium.Marker(
        location=coordinates,
        popup=name,
        icon=DivIcon(
                    icon_size=(20,20),
                    icon_anchor=(0,0),
                    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name,
                    )
    ).add_to(site_map)

# Show map
site_map

Now, you can explore the map by zoom-in/out the marked areas, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

***Findings:***

All of the launch sites are in coastal areas. VAFB SLC is the fartherst from the equator line.

### Task 2: Mark the success/failed launches for each site on the map

Next, let's enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.

Recall that dataframe spacex_df has detailed launch records, and the `class` column indicates if a launch was successful or not.

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.

If a launch was successful `(class=1)`, then we use a *green marker*, and if a launch failed `(class=0)`, we use a *red marker*.

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinates. *Marker clusters* can be a good way to simplify a map containing many markers with the same coordinates.

Let's first create a `MarkerCluster` object.

In [10]:
marker_cluster = MarkerCluster()

***TO DO:*** Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value.

In [11]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

launch_sites_df = spacex_df

def determine_marker_color(row):
    return 'green' if row['class'] == 1 else 'red'

spacex_df['marker_color'] = spacex_df.apply(determine_marker_color, axis=1)

launch_sites_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


***TO DO:*** For each launch result in `spacex_df` dataframe, add a `folium.Marker` to `marker_cluster`

In [12]:
from folium import plugins

# Create a MarkerCluster to hold the markers
marker_cluster = plugins.MarkerCluster().add_to(site_map)

# Function to determine marker color
def determine_marker_color(row):
    return 'green' if row['class'] == 1 else 'red'

# Add markers to the MarkerCluster
for index, row in spacex_df.iterrows():
    coordinates = [row['Lat'], row['Long']]
    name = row['Launch Site']
    marker_color = determine_marker_color(row)

    marker = folium.Marker(
        location=coordinates,
        popup=name,
        icon=folium.Icon(color='white', icon_color=marker_color)
    )
    marker.add_to(marker_cluster)

site_map

### TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinates for a mouse over a point on the map. This will help to easily find the coordinates of any point of interest (such as a railway), while you are exploring the map.

In [13]:
# Add Mouse Position to get the coordinates (Lat, Long) for a mouse over on the map

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

mouse_position.add_to(site_map)

site_map

Now, zoom in to a launch site and explore its proximities to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top left) in order to calculate the distance to the launch site.

In [14]:
#Auxiliary function to calculate distances

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

***TO DO:*** Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [15]:
# Launch sites coordinates
launch_sites = [
    {'name': 'CCAFS LC', 'lat1': 28.562302, 'lon1': -80.577356},
    {'name': 'CCAFS SLC', 'lat1': 28.563197, 'lon1': -80.576820},
    {'name': 'KSC LC', 'lat1': 28.573255, 'lon1': -80.646895},
    {'name': 'VAFB SLC', 'lat1': 34.632834, 'lon1': -120.610745}
]

# Find coordinates of the closet coastline (e.g.,: Lat: 28.56367  Lon: -80.57163):
coastline_points = [
    {'name': 'coast_coord_CCAFS LC', 'lat2': 28.56268, 'lon2': -80.56785},
    {'name': 'coast_coord_CCAFS SLC', 'lat2': 28.56268, 'lon2': -80.56785},
    {'name': 'coast_coord_KSC LC', 'lat2': 28.58264, 'lon2': -80.57399},
    {'name': 'coast_coord_VAFB SLC', 'lat2': 34.63293, 'lon2': -120.62611}
]

# Calculate the distance from the launch site to the coastline point: 
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Find coordinates of the closest coastline for each launch site:
for launch_site in launch_sites:
    launch_site_lat = launch_site['lat1']
    launch_site_lon = launch_site['lon1']

    # Find the corresponding coastline point
    coastline_point = next((point for point in coastline_points if point['name'] == f"coast_coord_{launch_site['name']}"), None)

    if coastline_point is not None:
        # Extracting coordinates from the dictionaries
        coastline_lat = coastline_point['lat2']
        coastline_lon = coastline_point['lon2']

        # Calculate the distance
        distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
        
        # Print or use the distance as needed
        print(f"The distance from {launch_site['name']} to its corresponding coastline is {distance_coastline} km")
    else:
        print(f"No matching coastline point found for {launch_site['name']}")

The distance from CCAFS LC to its corresponding coastline is 0.9296184400687078 km
The distance from CCAFS SLC to its corresponding coastline is 0.878184953864818 km
The distance from KSC LC to its corresponding coastline is 7.197347203425992 km
The distance from VAFB SLC to its corresponding coastline is 1.4062615925227433 km


***TO DO:*** Create and add a `folium.Marker` on your selected closest coastline point on the map. 

Display the distance between coastline point and launch site using the `icon` property. 

distance_marker = folium.Marker(
                            coordinate,
                            icon=DivIcon(
                                icon_size=(20,20),
                                icon_anchor=(0,0),
                                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

In [16]:
coastline_points = [
    {'name': 'coast_coord_CCAFS LC', 'coordinates': [28.56268, -80.56785], 'distance_km': 0.9296184400687078},
    {'name': 'coast_coord_CCAFS SLC', 'coordinates': [28.56268, -80.56785], 'distance_km': 0.878184953864818},
    {'name': 'coast_coord_KSC LC', 'coordinates': [28.58264, -80.57399], 'distance_km': 7.197347203425992},
    {'name': 'coast_coord_VAFB SLC', 'coordinates': [34.63293, -120.62611], 'distance_km': 1.4062615925227433}
]

# Add markers for coastline point
for point in coastline_points:
    name = point['name']
    coordinates = point['coordinates']
    popup = point['distance_km']
    
    # Add marker
    distance_marker = folium.Marker(
                            location=coordinates,
                            popup=popup,
                            icon=DivIcon(
                                icon_size=(20,20),
                                icon_anchor=(0,0),
                                html = '<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(point['distance_km'])


                            )        
                    ).add_to(site_map)

# Show map
site_map

[{'name': 'coast_coord_CCAFS LC', 'coordinates': [28.56268, -80.56785], 'distance_km': 0.9296184400687078}, {'name': 'coast_coord_CCAFS SLC', 'coordinates': [28.56268, -80.56785], 'distance_km': 0.878184953864818}, {'name': 'coast_coord_KSC LC', 'coordinates': [28.58264, -80.57399], 'distance_km': 7.197347203425992}, {'name': 'coast_coord_VAFB SLC', 'coordinates': [34.63293, -120.62611], 'distance_km': 1.4062615925227433}]
<class 'list'>


***TO DO:*** Draw a `PolyLine` between a launch site and a selected coastline point.

Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinates.

In [17]:
# Launch sites coordinates
launch_sites = [
    {'name': 'CCAFS LC', 'coordinates': [28.562302, -80.577356]},
    {'name': 'CCAFS SLC', 'coordinates': [28.563197, -80.576820]},
    {'name': 'KSC LC', 'coordinates': [28.573255, -80.646895]},
    {'name': 'VAFB SLC', 'coordinates': [34.632834, -120.610745]}
]

# Find coordinates of the closet coastline (e.g.,: Lat: 28.56367  Lon: -80.57163):
coastline_points = [
    {'name': 'coast_coord_CCAFS LC', 'coordinates': [28.56268, -80.56785]},
    {'name': 'coast_coord_CCAFS SLC', 'coordinates': [28.56268, -80.56785]},
    {'name': 'coast_coord_KSC LC', 'coordinates': [28.58264, -80.57399]},
    {'name': 'coast_coord_VAFB SLC', 'coordinates': [34.63293, -120.62611]}
]

In [18]:
# Find coordinates of the closest coastline for each launch site:
for launch_site in launch_sites:
    site_coordinates = launch_site['coordinates']
    
    # Find the corresponding coastline point
    coastline_point = next((point for point in coastline_points if point['name'] == f"coast_coord_{launch_site['name']}"), None)

    if coastline_point is not None:
        # Extracting coordinates from the dictionaries
        coastline_coordinates = coastline_point['coordinates']

        # Calculate the distance
        lines = folium.PolyLine(locations=[site_coordinates, coastline_coordinates], weight=1)
        
        lines.add_to(site_map)

site_map

***TODO:*** Similarly, you can draw a line between a launch site and its closest city, railway, highway, etc.

You need to use `MousePosition` to find the their coordinates on the map first.

Create a marker with distance to a closest city, railway, highway, etc.

Draw a line between the marker to the launch site.

In [27]:
# List of proximities to launch sites:
proximities = [
    {'category': 'City', 'name': 'CCAFS LC', 'coordinates': [28.6122, -80.8076]},
    {'category': 'City', 'name': 'CCAFS SLC', 'coordinates': [28.6122, -80.8076]},
    {'category': 'City', 'name': 'KSC LC', 'coordinates': [28.6122, -80.8076]},
    {'category': 'City', 'name': 'VAFB SLC', 'coordinates': [34.639, -120.45788]},

    {'category': 'Railroad', 'name': 'CCAFS LC', 'coordinates': [28.57103, -80.58542]},
    {'category': 'Railroad', 'name': 'CCAFS SLC', 'coordinates': [28.57103, -80.58542]},
    {'category': 'Railroad', 'name': 'KSC LC', 'coordinates': [28.573255, -80.65397]},
    {'category': 'Railroad', 'name': 'VAFB SLC', 'coordinates': [34.632834, -120.6249]},

    {'category': 'Highway', 'name': 'CCAFS LC', 'coordinates': [28.56255, -80.58708]},
    {'category': 'Highway', 'name': 'CCAFS SLC', 'coordinates': [28.56255, -80.58708]},
    {'category': 'Highway', 'name': 'KSC LC', 'coordinates': [28.573255, -80.65553]},
    {'category': 'Highway', 'name': 'VAFB SLC', 'coordinates': [34.68291, -120.60396]}
]

# Distances from launch sites to corresponding proximities:
for launch_site in launch_sites:
    launch_site_lat = launch_site['lat1']
    launch_site_lon = launch_site['lon1']

    # Find the corresponding proximity points for all categories
    proximities_for_site = [proximity for proximity in proximities if launch_site['name'] == proximity['name']]

    for proximity in proximities_for_site:
        # Extracting coordinates from the dictionaries
        proximity_lat = proximity['coordinates'][0]
        proximity_lon = proximity['coordinates'][1]

        # distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

        # Calculate the distance
        distance_proximity = calculate_distance(launch_site_lat, launch_site_lon, proximity_lat, proximity_lon)

        # Print or use the distance as needed
        print(f"The distance from {launch_site['name']} to closest {proximity['category']} is {distance_proximity} km")

The distance from CCAFS LC to closest City is 23.16264945461472 km
The distance from CCAFS LC to closest Railroad is 1.2502200965311898 km
The distance from CCAFS LC to closest Highway is 0.9503651725373442 km
The distance from CCAFS SLC to closest City is 23.189785230394932 km
The distance from CCAFS SLC to closest Railroad is 1.21033151561547 km
The distance from CCAFS SLC to closest Highway is 1.0049040350747516 km
The distance from KSC LC to closest City is 16.28201763469418 km
The distance from KSC LC to closest Railroad is 0.6911054091185059 km
The distance from KSC LC to closest Highway is 0.8434904886658036 km
The distance from VAFB SLC to closest City is 14.006654667267705 km
The distance from VAFB SLC to closest Railroad is 1.2954813214528493 km
The distance from VAFB SLC to closest Highway is 5.604432220906893 km


In [38]:
markers_list = [
    {'name': 'City closest to CCAFS LC', 'coordinates': [28.6122, -80.8076], 'distance': 23.16264945461472},
    {'name': 'City closest to CCAFS SLC', 'coordinates': [28.6122, -80.8076], 'distance': 23.189785230394932},
    {'name': 'City closest to KSC LC', 'coordinates': [28.6122, -80.8076], 'distance': 16.28201763469418},
    {'name': 'City closest to VAFB SLC', 'coordinates': [34.639, -120.45788], 'distance': 14.006654667267705},

    {'name': 'Railroad closest to CCAFS LC', 'coordinates': [28.57103, -80.58542], 'distance': 1.2502200965311898},
    {'name': 'Railroad closest to CCAFS SLC', 'coordinates': [28.57103, -80.58542], 'distance': 1.21033151561547},
    {'name': 'Railroad closest to KSC LC', 'coordinates': [28.573255, -80.65397], 'distance': 0.6911054091185059},
    {'name': 'Railroad closest to VAFB SLC', 'coordinates': [34.632834, -120.6249], 'distance': 1.2954813214528493},

    {'name': 'Highway closest to CCAFS LC', 'coordinates': [28.56255, -80.58708], 'distance': 0.9503651725373442},
    {'name': 'Highway closest to CCAFS SLC', 'coordinates': [28.56255, -80.58708], 'distance': 1.0049040350747516},
    {'name': 'Highway closest to KSC LC', 'coordinates': [28.573255, -80.65553], 'distance': 0.8434904886658036},
    {'name': 'Highway closest to VAFB SLC', 'coordinates': [34.68291, -120.60396], 'distance': 5.604432220906893}
]

# Add markers for proximities
for marker in markers_list:
    name = marker['name']
    coordinates = marker['coordinates']
    distance = "{:.2f} KM".format(marker['distance'])  # Ensure distance is a string
    
    # Add marker
    distance_marker = folium.Marker(
                            location=coordinates,
                            popup='<b>{}</b><br>Distance: {}'.format(name, distance),
                            icon=DivIcon(
                                icon_size=(20,20),
                                icon_anchor=(0,0),
                                html='<div style="font-size: 12px; color:#d35400;"><b>{}</b></div>'.format(distance)

                            )        
                    ).add_to(site_map)
site_map

In [43]:
# Draw a PolyLine between a launch site and proximities.

for category in set(proximity['category'] for proximity in proximities):
    for launch_site in launch_sites:
        site_coordinates = launch_site['coordinates']
        
        # Find proximities for the current launch site and category
        proximities_for_site = [proximity for proximity in proximities if
                                proximity['name'] == launch_site['name'] and proximity['category'] == category]

        for proximity in proximities_for_site:
            proximity_coordinates = proximity['coordinates']

            # Calculate the distance
            lines = folium.PolyLine(locations=[site_coordinates, proximity_coordinates], weight=1)
            lines.add_to(site_map)

site_map